In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!pip install tensorflow-addons

In [20]:
SEED = 62


import os
import re
import gc
import h5py
import torch
import string
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tqdm import tqdm
from nltk import tokenize

from sklearn import preprocessing
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation #, merge
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPooling2D
from keras.layers import RepeatVector, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import cross_val_score
from keras.initializers import Constant

def reset_seeds():
    np.random.seed(SEED)
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.python.keras.layers import Layer, InputSpec, Lambda
# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [21]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')

    feature_names = list(read_file.keys())
    loaded_data = []

    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [22]:
def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('/content/drive/MyDrive/MVSA_features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))

    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)

    return mvsa_single, mvsa_multiple

def load_mvsa_texts(merge=False):
    folder_path = '/content/drive/MyDrive/mvsa-data'
    file_paths = os.listdir(folder_path)
    for path in file_paths:
        file_name = os.path.split(path)[1]
        if file_name.split('.')[1] == 'hdf5':
            if file_name.split('-')[1] == 'single':
                mvsa_single_texts_path = os.path.join(folder_path, path)
            else:
                mvsa_multiple_texts_path = os.path.join(folder_path, path)

    mvsa_single = read_hdf5(mvsa_single_texts_path)
    mvsa_multiple = read_hdf5(mvsa_multiple_texts_path)

    for x in mvsa_single:
        if x[0] == 'texts':
            mvsa_single_texts = x[1]

    for x in mvsa_multiple:
        if x[0] == 'texts':
            mvsa_multiple_texts = x[1]

    if merge == True:
        return merge_mvsa(mvsa_single_texts, mvsa_multiple_texts)

    return mvsa_single_texts, mvsa_multiple_texts

def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]

    return labels, text_labels, image_labels

In [23]:
def clean_text(txt):
#     nonEnglish_regex = re.compile('[^a-zA-Z0-9\\?\\!\\,\\.@#\\+\\-=\\*\'\"><&\\$%\\(\\)\\[\\]:;]+')
#     hashtag_pattern = re.compile('#[a-zA-Z0-9]+')
#     txt_hashtag = re.sub(hashtag_pattern, '', txt)
    at_pattern = re.compile('@[a-zA-Z0-9]+')
    http_pattern = re.compile("((http|ftp|https)://)(([a-zA-Z0-9\._-]+\.[a-zA-Z]{2,6})|([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}))(:[0-9]{1,4})*(/[a-zA-Z0-9\&%_\./-~-]*)?")
    punc_pattern = re.compile('[%s]' % re.escape(string.punctuation))
    txt = re.sub('#', '', txt)
    txt = re.sub(at_pattern, 'user', txt)
    txt = re.sub(http_pattern, 'link', txt)
    txt = re.sub(punc_pattern, '', txt) # only remove punc for word2vec not BERT

    # as the data was crawled using Twitter API, it marked retweet data with RT <user> tag which has no meaning considering it in training
    if txt.startswith('RT user'):
        txt = ''.join(txt.split(':')[1:])
    txt = txt.strip()

    return txt

def get_clean_texts(data):
    cleaned_data = []
    for line in data:
        text = clean_text(line)
        cleaned_data.append(text)
    return cleaned_data

In [24]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 4, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 3)
    plt.plot(history.history['f1_macro'])
    plt.plot(history.history['val_f1_macro'])
    plt.title('Macro F1-SCORE')
    plt.ylabel('f1-macro')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 4)
    plt.plot(history.history['f1_weighted'])
    plt.plot(history.history['val_f1_weighted'])
    plt.title('Weighted F1-SCORE')
    plt.ylabel('f1-weighted')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [25]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * data.shape[0])
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

In [26]:
def create_model_text(input_shape):
    f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
    f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')

    text_input = Input(shape=input_shape)
    dropout = Dropout(DROPOUT_INPUT) (text_input)
    reshape_text = Reshape((1, -1)) (dropout)
    lstm = LSTM(NUM_LSTM) (reshape_text)
    dropout = Dropout(DROPOUT_LSTM) (lstm)

    outputs = Dense(NUM_CLASSES, activation='softmax') (dropout)

    model = Model(text_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_macro, f1_weighted])

    return model

In [27]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
    if checkpoint is not None:
        model = load_model('./model_checkpoint/{}.h5'.format(checkpoint))

    loss, acc, f1_macro, f1_weighted = model.evaluate(X_test, y_test, verbose=verbose)

    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('Macro F1-score:', f1_macro)
        print('Weighted F1-score:', f1_weighted)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(le.inverse_transform(y_test.argmax(axis=1)), le.inverse_transform(y_pred.argmax(axis=1)),
                                  labels=list(le.classes_))
        cm_disp = ConfusionMatrixDisplay(confusion_matrix=matrix,
                                  display_labels=list(le.classes_))
        cm_disp.plot()
        plt.show()

    return loss, acc, f1_macro, f1_weighted

In [28]:
def run_and_evaluate(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))

    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)

    if 'multiple' in name:
        batch_size = BATCH_SIZE_MULTIPLE
        if DO_SMOTE_MULTIPLE == True:
            oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
            #oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
            X_train, y_train = oversample.fit_resample(X_train, y_train)
    else:
        batch_size = BATCH_SIZE_SINGLE # 128
        if DO_SMOTE_SINGLE == True:
#             oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
            oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
            X_train, y_train = oversample.fit_resample(X_train, y_train)

    model = create_model_text(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=PATIENCE)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=PATIENCE)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    print(model.summary())
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                        callbacks=[checkpoint, early_stopping,reduce_lr])
    best_epoch = np.argmin(history.history['val_loss'])
    print('Checkpoint loaded at epoch:', best_epoch)

    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [29]:
def run_and_evaluate_w2v(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))

    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)

    if 'multiple' in name:
        batch_size = BATCH_SIZE_MULTIPLE
        if DO_SMOTE_MULTIPLE == True:
#             oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
            oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
            X_train, y_train = oversample.fit_resample(X_train, y_train)
    else:
        batch_size = BATCH_SIZE_SINGLE # 128
        if DO_SMOTE_SINGLE == True:
#             oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
            oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
            X_train, y_train = oversample.fit_resample(X_train, y_train)

    model = create_model_text_w2v(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=PATIENCE)
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=PATIENCE)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)

    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                        callbacks=[checkpoint, early_stopping])

    best_epoch = np.argmin(history.history['val_loss'])
    print('Checkpoint loaded at epoch:', best_epoch)

    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [30]:
def style_dataframe(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:lawngreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:tomato', axis=0)\
                          .highlight_min(subset=['Loss'], props='color:lawngreen', axis=0)\
                          .highlight_max(subset=['Loss'], props='color:tomato', axis=0)
def style_dataframe_out(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-weighted'], props='color:lawngreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-weighted'], props='color:tomato', axis=0)

In [31]:
from IPython.display import display_html

def display_dataframes(dfs, names=[], index=False):
    def to_df(x):
        if isinstance(x, pd.Series):
            return pd.DataFrame(x)
        else:
            return x
    html_str = ''
    if names:
        html_str += ('<tr>' +
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) +
                     '</tr>')
    html_str += ('<tr>' +
                 ''.join(f'<td style="vertical-align:top"> {to_df(df).to_html()}</td>'
                         for df in dfs) +
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)
    return html_str

# Load GloVe

In [32]:
MAX_LENGTH = 280 # as in twitter'S word limit
path_to_glove_file = '/content/drive/MyDrive/glove.twitter.27B.100d.txt'

In [33]:
mvsa_single_texts, mvsa_multiple_texts = load_mvsa_texts()

mvsa_single_texts = get_clean_texts(mvsa_single_texts)
mvsa_multiple_texts = get_clean_texts(mvsa_multiple_texts)

In [34]:
mvsa_single_texts[1]

'grattis min griskulting va bara tvungen oki s sch  user pig happybday wow lovely cut¡\xad'

In [35]:
from keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=MAX_LENGTH)

text_ds_single = tf.data.Dataset.from_tensor_slices(mvsa_single_texts).batch(128)
text_ds_multiple = tf.data.Dataset.from_tensor_slices(mvsa_multiple_texts).batch(128)

vectorizer.adapt(text_ds_single)
vectorizer.adapt(text_ds_multiple)

In [36]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [37]:
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 1193514 word vectors.


In [38]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

embedding_layer_glove = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer= Constant(embedding_matrix),
    trainable=False,
)

def create_model_text_w2v(input_shape):
    f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
    f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')

    text_input = Input(shape=(None,), dtype="int64")
    embedded_sequences = embedding_layer_glove(text_input)
    lstm = LSTM(NUM_LSTM) (embedded_sequences)
    lstm = Dropout(DROPOUT_LSTM) (lstm)
    outputs = Dense(NUM_CLASSES, activation='softmax') (lstm)

    model = Model(text_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_macro, f1_weighted])

    return model

Converted 12712 words (7288 misses)


# Load features data

In [39]:
mvsa_single_texts = vectorizer(np.array([[s] for s in mvsa_single_texts])).numpy()
mvsa_multiple_texts = vectorizer(np.array([[s] for s in mvsa_multiple_texts])).numpy()

In [40]:
mvsa_texts = merge_mvsa(mvsa_single_texts, mvsa_multiple_texts)

In [41]:
mvsa_single_bert_base, mvsa_multiple_bert_base = load_mvsa_feature('bert-base')
#mvsa_single_pos_bow, mvsa_multiple_pos_bow = load_mvsa_feature('pos-bow')
#mvsa_single_pos_tfidf, mvsa_multiple_pos_tfidf = load_mvsa_feature('pos-tfidf')
#mvsa_single_ner_bow, mvsa_multiple_ner_bow = load_mvsa_feature('ner-bow')
#mvsa_single_ner_tfidf, mvsa_multiple_ner_tfidf = load_mvsa_feature('ner-tfidf')

In [ ]:
mvsa_bert_base = merge_mvsa(mvsa_single_bert_base, mvsa_multiple_bert_base)
#mvsa_pos_bow = merge_mvsa(mvsa_single_pos_bow, mvsa_multiple_pos_bow)
#mvsa_pos_tfidf = merge_mvsa(mvsa_single_pos_tfidf, mvsa_multiple_pos_tfidf)
#mvsa_ner_bow = merge_mvsa(mvsa_single_ner_bow, mvsa_multiple_ner_bow)
#mvsa_ner_tfidf = merge_mvsa(mvsa_single_ner_tfidf, mvsa_multiple_ner_tfidf)

In [ ]:
mvsa_single_multimodal_labels, mvsa_single_text_labels, _ = load_labels('/content/drive/MyDrive/MVSA_features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, mvsa_multiple_text_labels, _ = load_labels('/content/drive/MyDrive/MVSA_features/labels/mvsa-multiple-labels.hdf5')

mvsa_multimodal_labels = merge_mvsa(mvsa_single_multimodal_labels, mvsa_multiple_multimodal_labels)
mvsa_text_labels = merge_mvsa(mvsa_single_text_labels, mvsa_multiple_text_labels)
# mvsa_image_labels = merge_mvsa(mvsa_single_image_labels, mvsa_multiple_image_labels)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(mvsa_multimodal_labels)
NUM_CLASSES = len(le.classes_) # =3
NUM_CLASSES

3

In [ ]:
def shuffle_mvsa(mvsa_features, labels, indices):
    shuffled_features = []
#     random_idx = np.random.permutation(len(labels))
    for i in range(len(mvsa_features)):
        x = mvsa_features[i][indices]
        shuffled_features.append(x)
    return shuffled_features, labels[indices]

In [ ]:
feature_names = ['glove',
                 'bert-base']
#                  'bert-pos-bow',
#                  'bert-pos-ner-bow',
                 #'bert-pos-tfidf',
                 #'bert-pos-ner-tfidf']

mvsa_single_features = [mvsa_single_texts,
                        mvsa_single_bert_base]
#                         np.concatenate((mvsa_single_bert_base, mvsa_single_pos_bow), axis=1),
#                         np.concatenate((mvsa_single_bert_base, mvsa_single_pos_bow, mvsa_single_ner_bow), axis=1),
                        #np.concatenate((mvsa_single_bert_base, mvsa_single_pos_tfidf), axis=1),
                        #np.concatenate((mvsa_single_bert_base, mvsa_single_pos_tfidf, mvsa_single_ner_tfidf), axis=1)]

mvsa_multiple_features = [mvsa_multiple_texts,
                          mvsa_multiple_bert_base]
#                           np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_bow), axis=1),
#                           np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_bow, mvsa_multiple_ner_bow), axis=1),
                          #np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_tfidf), axis=1),
                          #np.concatenate((mvsa_multiple_bert_base, mvsa_multiple_pos_tfidf, mvsa_multiple_ner_tfidf), axis=1)]

# mvsa_features = [mvsa_texts,
#                  mvsa_bert_base,
#                  np.concatenate((mvsa_bert_base, mvsa_pos_bow), axis=1),
#                  np.concatenate((mvsa_bert_base, mvsa_ner_bow), axis=1),
#                  np.concatenate((mvsa_bert_base, mvsa_pos_bow, mvsa_ner_bow), axis=1),
#                  np.concatenate((mvsa_bert_base, mvsa_pos_tfidf), axis=1),
#                  np.concatenate((mvsa_bert_base, mvsa_ner_tfidf), axis=1),
#                  np.concatenate((mvsa_bert_base, mvsa_pos_tfidf, mvsa_ner_tfidf), axis=1)]

# Fix random indices for consistency between other experiments
#mvsa_single_features, mvsa_single_multimodal_labels = shuffle_mvsa(mvsa_single_features, mvsa_single_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-single-shuffle-indices.npy'))
#mvsa_multiple_features, mvsa_multiple_multimodal_labels = shuffle_mvsa(mvsa_multiple_features, mvsa_multiple_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-multiple-shuffle-indices.npy'))

In [ ]:
mvsa_single_bert_base

array([[ 7.5397608e-03,  1.8611285e-01, -6.9106869e-02, ...,
        -5.0248194e-04,  1.4276108e-01,  2.4909839e-02],
       [-3.8780382e-01, -6.0706139e-02,  2.6096505e-01, ...,
        -1.7109679e-01,  4.0860008e-02,  4.6039146e-01],
       [ 1.6086867e-02,  2.8751701e-02,  5.4987099e-02, ...,
        -1.6268755e-01,  1.6949457e-01,  2.1895787e-01],
       ...,
       [-6.8853624e-02,  2.1740802e-01,  1.1597792e-01, ...,
        -3.6182353e-01,  3.4161410e-01,  2.1755539e-01],
       [ 4.3439984e-02, -4.6166394e-02,  1.3135825e-01, ...,
        -3.3200899e-01,  4.0010512e-02,  5.3713363e-01],
       [-1.7515238e-01,  1.2460667e-01,  4.8288941e-02, ...,
        -7.0521541e-02,  5.6701380e-01,  4.4989815e-01]], dtype=float32)

# Run models and Evalution display

In [ ]:
reset_seeds()
EPOCHS = 100
VALIDATION_SPLIT = 0.1
PATIENCE = 10

BATCH_SIZE_SINGLE = 128
BATCH_SIZE_MULTIPLE = 256

DO_SMOTE_SINGLE = True
DO_SMOTE_MULTIPLE = True

NUM_LSTM = 256
DROPOUT_INPUT = 0.0
DROPOUT_LSTM = 0.9

OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'

## With original text labels

In [ ]:
# print('MVSA-Single: With original text labels\n')
# mvsa_single_histories = []
# mvsa_single_scores = []
# for i in range(len(feature_names)):
#     print('MVSA-Single:', feature_names[i])
#     if feature_names[i] == 'glove':
#         history, score = run_and_evaluate_w2v('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_text_labels, verbose=0)
#     else:
#         history, score = run_and_evaluate('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_text_labels, verbose=0)
#     mvsa_single_histories.append(history)
#     mvsa_single_scores.append(score)
#     print()
# df_single_scores = pd.DataFrame(mvsa_single_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# print('MVSA-Multiple: With original text labels\n')
# mvsa_multiple_histories = []
# mvsa_multiple_scores = []
# for i in range(len(feature_names)):
#     print('MVSA-Multiple:', feature_names[i])
#     if feature_names[i] == 'glove':
#         history, score = run_and_evaluate_w2v('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_text_labels, verbose=0)
#     else:
#         history, score = run_and_evaluate('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_text_labels, verbose=0)
#     mvsa_multiple_histories.append(history)
#     mvsa_multiple_scores.append(score)
#     print()
# df_multiple_scores = pd.DataFrame(mvsa_multiple_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# mvsa_average_scores = np.mean([mvsa_single_scores, mvsa_multiple_scores], axis=0)
# df_average_scores = pd.DataFrame(mvsa_average_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

In [ ]:
len(mvsa_multiple_features[1]), len(mvsa_multiple_multimodal_labels)

(17024, 17024)

## With multimodal labels

In [ ]:
print('MVSA-Single: With multimodal labels')
mvsa_single_histories2 = []
mvsa_single_scores2 = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    history, score = run_and_evaluate('single-ML-' + feature_names[i], mvsa_single_features[i], mvsa_single_multimodal_labels, verbose=0)
    mvsa_single_histories2.append(history)
    mvsa_single_scores2.append(score)
    print()
df_single_scores2 = pd.DataFrame(mvsa_single_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('MVSA-Multiple: With multimodal labels')
mvsa_multiple_histories2 = []
mvsa_multiple_scores2 = []
for i in range(len(feature_names)):
   print('MVSA-Multiple:', feature_names[i])
   history, score = run_and_evaluate('multiple-ML-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_multimodal_labels, verbose=0)
   mvsa_multiple_histories2.append(history)
   mvsa_multiple_scores2.append(score)
   print()
df_multiple_scores2 = pd.DataFrame(mvsa_multiple_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

mvsa_average_scores2 = np.mean([mvsa_single_scores2, mvsa_multiple_scores2], axis=0)
df_average_scores2 = pd.DataFrame(mvsa_average_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single: With multimodal labels
MVSA-Single: glove
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280)]             0         
                                                                 
 dropout (Dropout)           (None, 280)               0         
                                                                 
 reshape (Reshape)           (None, 1, 280)            0         
                                                                 
 lstm (LSTM)                 (None, 256)               549888    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 3)                 771       
                                                                 
Total 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Checkpoint loaded at epoch: 20

MVSA-Single: bert-base
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 768)]             0         
                                                                 
 dropout_2 (Dropout)         (None, 768)               0         
                                                                 
 reshape_1 (Reshape)         (None, 1, 768)            0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               1049600   
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Tota

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Checkpoint loaded at epoch: 12

MVSA-Multiple: With multimodal labels
MVSA-Multiple: glove
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 280)]             0         
                                                                 
 dropout_4 (Dropout)         (None, 280)               0         
                                                                 
 reshape_2 (Reshape)         (None, 1, 280)            0         
                                                                 
 lstm_2 (LSTM)               (None, 256)               549888    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 771       
                                  

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Checkpoint loaded at epoch: 17

MVSA-Multiple: bert-base
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 768)]             0         
                                                                 
 dropout_6 (Dropout)         (None, 768)               0         
                                                                 
 reshape_3 (Reshape)         (None, 1, 768)            0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               1049600   
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 3)                 771       
                                                                 
To

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Checkpoint loaded at epoch: 9



# With merge MVSA data

In [ ]:
# print('Both MVSA: With original image labels')
# mvsa_histories3 = []
# mvsa_scores3 = []
# for i in range(len(feature_names)):
#     print('Both MVSA:', feature_names[i])
#     history, score = run_and_evaluate('merge-OL-' + feature_names[i], mvsa_features[i], mvsa_text_labels, verbose=0)
#     mvsa_histories3.append(history)
#     mvsa_scores3.append(score)
#     print()
# df_scores3 = pd.DataFrame(mvsa_scores3, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# print('Both MVSA: With multimodal labels')
# mvsa_histories4 = []
# mvsa_scores4 = []
# for i in range(len(feature_names)):
#     print('Both MVSA:', feature_names[i])
#     history, score = run_and_evaluate('merge-ML-' + feature_names[i], mvsa_features[i], mvsa_multimodal_labels, verbose=0)
#     mvsa_histories4.append(history)
#     mvsa_scores4.append(score)
#     print()
# df_scores4 = pd.DataFrame(mvsa_scores4, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

# Display results

In [ ]:
# print('With original image labels\n')
# display_dataframes((style_dataframe(df_single_scores), style_dataframe(df_multiple_scores), style_dataframe(df_average_scores)),
#                    names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

In [ ]:
print('With multimodal labels\n')
display_dataframes((style_dataframe(df_single_scores2), style_dataframe(df_multiple_scores2), style_dataframe(df_average_scores2)),
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])
print()

With multimodal labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 glove 
 1.118690 
 0.478936 
 0.269708 
 0.416550 
 
 
 bert-base 
 1.007108 
 0.547672 
 0.428280 
 0.588598 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 glove 
 1.080586 
 0.437133 
 0.279994 
 0.423148 
 
 
 bert-base 
 0.887035 
 0.603995 
 0.384558 
 0.538420 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 glove 
 1.099638 
 0.458034 
 0.274851 
 0.419849 
 
 
 bert-base 
 0.947072 
 0.575834 
 0.406419 
 0.563509

In [ ]:
drop_columns = ['Loss', 'F1-macro']
def dataframe_to_display(df):
    return style_dataframe_out(np.round(df, 3).drop(columns=drop_columns))

In [ ]:
if not os.path.exists('./tables'):
    os.makedirs('./tables')
open('./tables/single_text_scores.html', 'w').write(dataframe_to_display(df_single_scores2).to_html())
#open('./tables/multiple_text_scores.html', 'w').write(dataframe_to_display(df_multiple_scores2).to_html())
#open('./tables/average_text_scores.html', 'w').write(dataframe_to_display(df_average_scores2).to_html())

942

In [ ]:
# print('With both MVSA merged together\n')
# display_dataframes((style_dataframe(df_scores4)),
#                    names=['Multimodal labels'])
# style_dataframe(df_scores4)

# Plots

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
WIDTH = 650
HEIGHT = 400
os.makedirs('./plots')

In [ ]:
df_single = pd.concat([df_single_scores2['Accuracy'], df_single_scores2['F1-weighted']], axis=1)
df_single['Đặc trưng'] = df_single.index
df_single['Bộ dữ liệu'] = 'MVSA-Single'
df_multiple = pd.concat([df_multiple_scores2['Accuracy'], df_multiple_scores2['F1-weighted']], axis=1)
df_multiple['Đặc trưng'] = df_multiple.index
df_multiple['Bộ dữ liệu'] = 'MVSA-Multiple'
df = pd.concat([df_single, df_multiple])

In [ ]:
import plotly.express as px

fig = px.bar(df, x="Đặc trưng", y=["Accuracy", "F1-weighted"], facet_col="Bộ dữ liệu", facet_col_spacing = 0.15)
fig.update_layout(barmode = 'group')
# fig.update_layout(bargap=0.2)
fig.update_layout(margin_b=25, margin_t=50, margin_l=25, margin_r=25, width=WIDTH, height=HEIGHT)
fig.update_layout(title='Biểu đồ kết quả đánh giá các đặc trưng của dữ liệu văn bản',
              yaxis_title='Điểm', legend_title='Độ đo')
#fig.write_html('./plots/text-scores-plot.html')
fig.show()

In [ ]:
# import plotly.graph_objects as go
# import plotly.express as px
# from plotly.subplots import make_subplots

# fig_single = px.bar(df_single, x="Features", y=["Accuracy", "F1-weighted"])
# fig_single.update_layout(barmode = 'group')
# fig_single.update_yaxes(range=[0, 0.7])
# fig_multiple = px.bar(df_multiple, x="Features", y=["Accuracy", "F1-weighted"])
# fig_multiple.update_layout(barmode = 'group')


# fig = make_subplots(rows=1, cols=2)
# fig.add_trace(fig_single['data'][0], row=1, col=1)
# fig.add_trace(fig_single['data'][1], row=1, col=1)

# fig.add_trace(fig_multiple['data'][0], row=1, col=2)
# fig.add_trace(fig_multiple['data'][1], row=1, col=2)
# fig.update_yaxes(range=[0, 0.7])
# fig.update_yaxes(tick0=0.05, dtick=0.05)